# Notebook to explore the GIS data associated with Burial Mounds

The data is available as a series of GeoTIFF images that have embedded GIS data and
shape-files containing a set of GIS points. We need to read these and transform them
to consistent coordinates to be able to process them.

GeoTIFF images are tiff images that have embedded GIS data, usually giving their
coordinate system and the bounds of the image. This means we can work out how to
overlay other GIS data on them.   They can also have multiple layers of data but these
files only have a single image that is a scan of a Soviet era map of Bulgaria.

The Shape-files contain GIS data. In this case, a set of points that have been hand-labelled
from the maps. Each point corresponds to a symbol on the map that might be a burial mound.
Our task in this project is to train a model to find these symbols in the map. While the
shape-files contain multiple symbols, this project will concentrate on 'Hairy brown circle'
symbols.

This notebook does some initial exploration of the data, showing how to read the files
and transform the shapefile and geoTIFF data to the same coordinate system.

NOTE: paths found in this notebook are absolute Google Drive paths. User may need to change the paths specific to their Google Drive paths as Google Colab notebook only works with absolute paths to one's Google Drive directories.

# Google Colab Additional Steps
The next 2 steps are additional steps to be done when running in Google Colab. User may skip the steps if notebook is run locally.



In [1]:
#installing the following packages since when run in Google Colab, we have to install on each runtime
!pip install rasterio


In [2]:
#mounted my google drive where the dataset is located
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [3]:
#imported necessary packages
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import os
from shapely.geometry import box


In [4]:
shapefile_path = '/content/drive/MyDrive/Colab Notebooks/summer_internship/data/shapefiles/MapMounds4326.shp'
gdf = gpd.read_file(shapefile_path)
gdf.head()

DriverError: '/content/drive/MyDrive/Colab Notebooks/summer_internship/data/shapefiles/MapMounds4326.shp' does not exist in the file system, and is not recognized as a supported dataset name.

In [ ]:
gdf.iloc[0]

In [ ]:
gdf.MpSymbl.unique()

In [ ]:
gdf.MpSymbl.value_counts()

In [ ]:
# we will just keep the Hairy brown circles
gdf = gdf[gdf.MpSymbl == 'Hairy brown circle']

### Next step basically does this.

1. Iterates through the directory and read all image tiff files (MAPS), more accurately rasters. Raster consists of a matrix of cells (or pixels) organized into rows and columns (or a grid) where each cell contains a value representing information. Transformation of coordinate systems require raster image files and gis files/dataframes.

2. For every image of a map, the gis file (gdf) is transformed to match the coordinate system of the image.

3. The transformed gdf is then filtered. Retaining only the points within the current map image. So we will be left with a gdf that consists only of hairy brown circle points that meet the longitude latitude bounds of the current map image.

4. Filtered GDF are now plotted on top of the images and represented as red circles. Encircling the hairy brown circles.

In [ ]:
from rasterio.warp import transform_bounds
geotiff_dir = '/content/drive/MyDrive/Colab Notebooks/summer_internship/data/YambolGIS/Training32635'

for filename in os.listdir(geotiff_dir):
    if filename.endswith('.tif'):
        geotiff_path = os.path.join(geotiff_dir, filename)
        with rasterio.open(geotiff_path) as src:
            img_data = src.read()
            # transpose the image data to make it suitable for plotting
            img_data = np.transpose(img_data, (1, 2, 0))

            # transform the shapefile data into the coordinate system of this image
            gdf_t = gdf.to_crs(src.crs)
            print(filename, ':', src.bounds)
            bbox = box(src.bounds.left, src.bounds.bottom, src.bounds.right, src.bounds.top)
            # find the points that are inside the bounds of this image
            gdf_within_bounds = gdf_t[gdf_t.geometry.within(bbox)]
            print('points within bounds: ', gdf_within_bounds.shape[0])

            # now plot the image and shapefile data
            fig, ax = plt.subplots(figsize=(20, 20))
            ax.imshow(img_data, extent=[src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top])
            gdf_within_bounds.plot(ax=ax, facecolor='none', edgecolor='red')
            plt.show()
